In [1]:
import pandas as pd
import os
import json
import ast
import statistics as stat

# Define scene graph nodes and relations

In [2]:
# these are the anatomical locations that there are bbox coordinates extracted from the bbox pipeline
objects_withbbox = pd.read_csv('../semantics/objects_detectable_by_bbox_pipeline_v1.txt',sep=',',header=None)
objects_withbbox = objects_withbbox[0].tolist()
print('Number of object node types that the Bbox pipeline v1 can extract:', len(objects_withbbox))

# these are the anatomical locations that the NLP pipeline tried to extract from reports
objects_nlp = pd.read_csv('../semantics/objects_extracted_from_reports_v1.txt',sep=',',header=None)
objects_nlp = objects_nlp[0].tolist()
print('Number of object node types that the NLP pipeline v1 can extract:', len(objects_nlp))

# NOT all objects_nlp are in objects_withbbox
print('Objects with no bbox:',set(objects_nlp).difference(set(objects_withbbox)))

# NOT all objects_withbbox are in objects_nlp -- not mentioned much in report or currently not addressed by nlp pipeline
print('Objects not in CXR report knowledge graph:',set(objects_withbbox).difference(set(objects_nlp)))


print()
# these are the labels/attributes that could exist in the Chest ImaGenome dataset
labels = pd.read_csv('../semantics/attribute_relations_v1.txt',sep=',',header=None)
labels = labels[0].tolist()
print('Number of attributes that can be associated with CXR objects:', len(labels))

print()
# These are the object-object relations between sequential exams
comp_relations = pd.read_csv('../semantics/comparison_relations_v1.txt',sep=',',header=None)
comp_relations =comp_relations[0].tolist()
print('Number of comparison relations between anatomical locations across sequential exams:',len(comp_relations))

# Number of (elements from the cxr knowledge graph that can be considered as) 
# potential different 'labels' = 90 + 38 + 3 = 131 (reported in table 1 in paper)

Number of object node types that the Bbox pipeline v1 can extract: 36
Number of object node types that the NLP pipeline v1 can extract: 38
Objects with no bbox: {'neck', 'right breast', 'right arm', 'left shoulder', 'left breast', 'right shoulder', 'left chest wall', 'left arm', 'right chest wall'}
Objects not in CXR report knowledge graph: {'right cardiac silhouette', 'descending aorta', 'left cardiophrenic angle', 'left cardiac silhouette', 'left upper abdomen', 'right upper abdomen', 'right cardiophrenic angle'}

Number of attributes that can be associated with CXR objects: 90

Number of comparison relations between anatomical locations across sequential exams: 3


In [3]:
# All the nodes in the cxr knowledge graph for each scene graph are mapped to UMLS CUI
ont = json.load(open('../semantics/label_to_UMLS_mapping.json', 'r'))
ont['nci_IDs']['right lung']

'C0225706'

# Attribute relations statistics

In [4]:
# This table can be generated from the scene_graph silver dataset using utils/postprocessing/scenegraph_postprocessing.py
# Make sure path to file is correct on your local machine

attributes = pd.read_csv('../../../subset/scene_tabular/attribute_relations_tabular.txt',sep='\t')
print(attributes.shape)

(9468649, 20)


In [5]:
attributes.head()

,image_id,study_id,sent_loc,row_id,bbox,attribute,context,label_name,categoryID,sentence,comparison,temporal,laterality,severity,texture,bbox_coord_224,bbox_coord_original,region,relation,annot_id
0,d7bef063-28053f7a-f27dae40-4035348b-21a36d32,55916528,0.02,55916528|0.02,right lung,anatomicalfinding|yes|lung opacity,yes,lung opacity,anatomicalfinding,moderate pulmonary edema,[],[],[],[],[],"[39, 48, 100, 178]","[532, 395, 1364, 2169]","['left hilar structures', 'left lung', 'right ...",1,55916528|0.02|right lung|1|lung opacity
1,d7bef063-28053f7a-f27dae40-4035348b-21a36d32,55916528,0.02,55916528|0.02,right lung,anatomicalfinding|yes|pulmonary edema/hazy opa...,yes,pulmonary edema/hazy opacity,anatomicalfinding,moderate pulmonary edema,[],[],[],[],[],"[39, 48, 100, 178]","[532, 395, 1364, 2169]","['left hilar structures', 'left lung', 'right ...",1,55916528|0.02|right lung|1|pulmonary edema/haz...
2,d7bef063-28053f7a-f27dae40-4035348b-21a36d32,55916528,0.02,55916528|0.02,right lung,nlp|yes|abnormal,yes,abnormal,nlp,moderate pulmonary edema,[],[],[],[],[],"[39, 48, 100, 178]","[532, 395, 1364, 2169]","['left hilar structures', 'left lung', 'right ...",1,55916528|0.02|right lung|1|abnormal
3,d7bef063-28053f7a-f27dae40-4035348b-21a36d32,55916528,7.00,55916528|7,right lung,anatomicalfinding|yes|pneumothorax,yes,pneumothorax,anatomicalfinding,the pre existing basal right pneumothorax see...,"['comparison|yes|improved', 'comparison|yes|wo...",[],[],[],[],"[39, 48, 100, 178]","[532, 395, 1364, 2169]","['right lower lung zone', 'right lung']",1,55916528|7|right lung|1|pneumothorax
4,d7bef063-28053f7a-f27dae40-4035348b-21a36d32,55916528,7.00,55916528|7,right lung,nlp|yes|abnormal,yes,abnormal,nlp,the pre existing basal right pneumothorax see...,"['comparison|yes|improved', 'comparison|yes|wo...",[],[],[],[],"[39, 48, 100, 178]","[532, 395, 1364, 2169]","['right lower lung zone', 'right lung']",1,55916528|7|right lung|1|abnormal


In [6]:
print(attributes.shape)
# This are sentences where an attribute is extracted by the pipeline could not relate it to an anatomical location
attributes = attributes[attributes.bbox !='unknown'].reset_index(drop=True).copy()
print(attributes.shape)

(9468649, 20)
(9468639, 20)


In [7]:
# to report in paper
print('Number of unique images:', len(set(attributes.image_id)))
print('Number of unique reports:', len(set(attributes.study_id)))

Number of unique images: 242072
Number of unique reports: 217013


In [8]:
uniq_b2a = ['|'.join(x.split('|')[2:]) for x in attributes.annot_id]
uniq_b2a = set([x for x in uniq_b2a if 'unknown' not in x])
print('Number of unique bbox to attribute relations:', len(uniq_b2a))

Number of unique bbox to attribute relations: 1256


In [9]:
print('Number of unique attributes:',len(set(attributes.label_name)))

# Not all of these bboxes have object coordinates on the images
print('Number of unique anatomical locations that can be detected by the NLP pipeline:', len(set(attributes.bbox)))
print()

# CXR anatomies that the bbox pipeline does not handle yet
# These can be detected with NLP so are present in the cxr knowledge graph 
# hence they show up in the attributes field of the scene graphs
nobboxes = list(set(objects_nlp).difference(set(objects_withbbox)))
print('Number of locations that the Bbox pipeline cannot extract yet:', len(nobboxes))

# But many of these objects likely can be roughly and easily derived from the existing objects with some clinical heuristics
# e.g. right chest wall would just overlap right lung but with more margin.

print() 

# bbox objects with coordinates and also describe in reports (so are in the cxr knowledge graph) are:
objects = sorted(list(set(attributes.bbox).difference(set(nobboxes))))
print('Number of unique object types with bbox coordinates:', len(objects))
print()

print(objects)

Number of unique attributes: 76
Number of unique anatomical locations that can be detected by the NLP pipeline: 38

Number of locations that the Bbox pipeline cannot extract yet: 9

Number of unique object types with bbox coordinates: 29

['abdomen', 'aortic arch', 'cardiac silhouette', 'carina', 'cavoatrial junction', 'left apical zone', 'left clavicle', 'left costophrenic angle', 'left hemidiaphragm', 'left hilar structures', 'left lower lung zone', 'left lung', 'left mid lung zone', 'left upper lung zone', 'mediastinum', 'right apical zone', 'right atrium', 'right clavicle', 'right costophrenic angle', 'right hemidiaphragm', 'right hilar structures', 'right lower lung zone', 'right lung', 'right mid lung zone', 'right upper lung zone', 'spine', 'svc', 'trachea', 'upper mediastinum']


In [10]:
# Sort by unique image and sent_loc (order of sentences in a report)
attributes = attributes.sort_values(by=['image_id','sent_loc','bbox','label_name']).reset_index(drop=True).copy()
print(attributes.shape)

# keep last context for each label_name at each location bbox for each image
attributes = attributes.drop_duplicates(subset=['image_id','bbox','label_name'],keep='last').reset_index(drop=True).copy()
print(attributes.shape)

# Simplify calculation for just detecting affirmed attributes
# also if negation had failed, the relation would be affirmed -- so will calculate performance on affirmed relations only
attributes = attributes[attributes.context=='yes'].copy()

keep = ['study_id', 'image_id', 'bbox', 'attribute', 'bbox_coord_224','bbox_coord_original', 'relation', 'annot_id','label_name']
# Drop the duplicates - rolled relations to report level
attributes = attributes[keep].drop_duplicates().reset_index(drop=True).copy()
print(attributes.shape)

(9468639, 20)
(7305008, 20)
(4529803, 9)


In [11]:
# One problem that may be of particular interests for some researchers
lesion = attributes[(attributes.label_name=='lung lesion')&(attributes.relation==1)].reset_index(drop=True).copy()
print('Number of localized lesion annotation:', len(lesion))
print('Number of images of lung lesion annotations:', len(set(lesion.image_id)))

Number of localized lesion annotation: 37647
Number of images of lung lesion annotations: 14062


In [12]:
# Get the average number of objects (bboxes) and attributes (label_name) that a given cxr knowledge graph from report may describe
# This number is calculated from just what is described/implied in CXR report descriptions
# This calculation has NOT included the total number of actual anatomical objects detected from each image by the bbox pipeline
# However, in the "objects" field in each scene graph json -- all detected bbox objects are included.

keep = ['image_id','bbox','label_name','bbox_coord_224']
averages = attributes[keep].copy()
print(averages.shape)
# drop bboxes with no coordinates
averages = averages[averages.bbox_coord_224 != "[0, 0, 0, 0]"].reset_index(drop=True).copy()
print(averages.shape)
# group by image_id then get unique bboxes and attributes
averages['ave_bboxes'] = averages.groupby(['image_id'])['bbox'].transform(lambda x: '&&'.join(sorted(list(set(x)))))
averages['ave_attrs'] = averages.groupby(['image_id'])['label_name'].transform(lambda x: '&&'.join(sorted(list(set(x)))))
keep2 = ['image_id','ave_bboxes','ave_attrs']
averages = averages[keep2].drop_duplicates().reset_index(drop=True).copy()
print(averages.shape)
# Count number of bboxes and attributes then get average
averages['ave_bboxes'] = [len(x.split('&&')) for x in averages['ave_bboxes']]
averages['ave_attrs'] = [len(x.split('&&')) for x in averages['ave_attrs']]
print('Average number of objects (bbox) related to attributes per image:', stat.mean(averages['ave_bboxes'].tolist()))
print('Average number of attributes per image:',stat.mean(averages['ave_attrs'].tolist()))
attributes.head()

(4529803, 4)
(4326409, 4)
(240500, 3)
Average number of objects (bbox) related to attributes per image: 7.244432432432433
Average number of attributes per image: 5.020220374220374


,study_id,image_id,bbox,attribute,bbox_coord_224,bbox_coord_original,relation,annot_id,label_name
0,55245226,000046e4-e4d7f796-72c3dba4-8b67a485-0eea211d,neck,tubesandlines|yes|endotracheal tube,"[0, 0, 0, 0]","[0, 0, 0, 0]",1,55245226|5|neck|1|endotracheal tube,endotracheal tube
1,55245226,000046e4-e4d7f796-72c3dba4-8b67a485-0eea211d,trachea,tubesandlines|yes|endotracheal tube,"[109, 75, 125, 96]","[1484, 762, 1702, 1048]",1,55245226|5|trachea|1|endotracheal tube,endotracheal tube
2,55245226,000046e4-e4d7f796-72c3dba4-8b67a485-0eea211d,left lower lung zone,nlp|yes|abnormal,"[132, 99, 210, 132]","[1797, 1089, 2859, 1538]",1,55245226|6|left lower lung zone|1|abnormal,abnormal
3,55245226,000046e4-e4d7f796-72c3dba4-8b67a485-0eea211d,left lower lung zone,anatomicalfinding|yes|atelectasis,"[132, 99, 210, 132]","[1797, 1089, 2859, 1538]",1,55245226|6|left lower lung zone|1|atelectasis,atelectasis
4,55245226,000046e4-e4d7f796-72c3dba4-8b67a485-0eea211d,left lower lung zone,anatomicalfinding|yes|lobar/segmental collapse,"[132, 99, 210, 132]","[1797, 1089, 2859, 1538]",1,55245226|6|left lower lung zone|1|lobar/segmen...,lobar/segmental collapse


In [13]:
uniq_boxes = list(set(attributes.bbox))
uniq_boxes.sort()
uniq_attrs = list(set(attributes.attribute))
uniq_attrs.sort()
print(len(uniq_boxes))
print(len(uniq_attrs))
print(type(attributes.loc[0,'bbox_coord_224']))
# sanity check no neg coordinates
print(attributes[attributes.bbox_coord_224.str.contains('-')].shape) 

38
76
<class 'str'>
(0, 9)


In [14]:
# Tally the different object-attribute relations (affirmed and negated calculated separately)
# All NLP detected object and attribute (after above deduplication/roll up to study level) are included in this tally
tally = {'bbox':[], 'attribute':[], 'freq_nlp':[], 'freq_has_bbox':[]}
for box in uniq_boxes:
    print(box)
    for attr in uniq_attrs:
        print(attr)
        df = attributes[(attributes.bbox==box)&(attributes.attribute==attr)].reset_index(drop=True).copy()
        if len(df)>0:
            tally['bbox'].append(box)
            tally['attribute'].append(attr)
            tally['freq_nlp'].append(len(df))
            tally['freq_has_bbox'].append(len(df[df.bbox_coord_224 != '[0, 0, 0, 0]']))

abdomen
anatomicalfinding|yes|airspace opacity
anatomicalfinding|yes|atelectasis
anatomicalfinding|yes|bone lesion
anatomicalfinding|yes|bronchiectasis
anatomicalfinding|yes|calcified nodule
anatomicalfinding|yes|clavicle fracture
anatomicalfinding|yes|consolidation
anatomicalfinding|yes|costophrenic angle blunting
anatomicalfinding|yes|cyst/bullae
anatomicalfinding|yes|diaphragmatic eventration (benign)
anatomicalfinding|yes|elevated hemidiaphragm
anatomicalfinding|yes|enlarged cardiac silhouette
anatomicalfinding|yes|enlarged hilum
anatomicalfinding|yes|hernia
anatomicalfinding|yes|hydropneumothorax
anatomicalfinding|yes|hyperaeration
anatomicalfinding|yes|increased reticular markings/ild pattern
anatomicalfinding|yes|infiltration
anatomicalfinding|yes|linear/patchy atelectasis
anatomicalfinding|yes|lobar/segmental collapse
anatomicalfinding|yes|lung lesion
anatomicalfinding|yes|lung opacity
anatomicalfinding|yes|mass/nodule (not otherwise specified)
anatomicalfinding|yes|mediastinal

tubesandlines|yes|chest tube
tubesandlines|yes|endotracheal tube
tubesandlines|yes|enteric tube
tubesandlines|yes|ij line
tubesandlines|yes|intra-aortic balloon pump
tubesandlines|yes|mediastinal drain
tubesandlines|yes|picc
tubesandlines|yes|pigtail catheter
tubesandlines|yes|subclavian line
tubesandlines|yes|swan-ganz catheter
tubesandlines|yes|tracheostomy tube
carina
anatomicalfinding|yes|airspace opacity
anatomicalfinding|yes|atelectasis
anatomicalfinding|yes|bone lesion
anatomicalfinding|yes|bronchiectasis
anatomicalfinding|yes|calcified nodule
anatomicalfinding|yes|clavicle fracture
anatomicalfinding|yes|consolidation
anatomicalfinding|yes|costophrenic angle blunting
anatomicalfinding|yes|cyst/bullae
anatomicalfinding|yes|diaphragmatic eventration (benign)
anatomicalfinding|yes|elevated hemidiaphragm
anatomicalfinding|yes|enlarged cardiac silhouette
anatomicalfinding|yes|enlarged hilum
anatomicalfinding|yes|hernia
anatomicalfinding|yes|hydropneumothorax
anatomicalfinding|yes|hyp

disease|yes|interstitial lung disease
disease|yes|lung cancer
disease|yes|pericardial effusion
disease|yes|pneumonia
nlp|yes|abnormal
nlp|yes|normal
technicalassessment|yes|artifact
technicalassessment|yes|breast/nipple shadows
technicalassessment|yes|low lung volumes
technicalassessment|yes|rotated
technicalassessment|yes|skin fold
tubesandlines|yes|chest port
tubesandlines|yes|chest tube
tubesandlines|yes|endotracheal tube
tubesandlines|yes|enteric tube
tubesandlines|yes|ij line
tubesandlines|yes|intra-aortic balloon pump
tubesandlines|yes|mediastinal drain
tubesandlines|yes|picc
tubesandlines|yes|pigtail catheter
tubesandlines|yes|subclavian line
tubesandlines|yes|swan-ganz catheter
tubesandlines|yes|tracheostomy tube
left arm
anatomicalfinding|yes|airspace opacity
anatomicalfinding|yes|atelectasis
anatomicalfinding|yes|bone lesion
anatomicalfinding|yes|bronchiectasis
anatomicalfinding|yes|calcified nodule
anatomicalfinding|yes|clavicle fracture
anatomicalfinding|yes|consolidation
a

anatomicalfinding|yes|vascular congestion
anatomicalfinding|yes|vascular redistribution
device|yes|aortic graft/repair
device|yes|cabg grafts
device|yes|cardiac pacer and wires
device|yes|prosthetic valve
disease|yes|alveolar hemorrhage
disease|yes|aspiration
disease|yes|copd/emphysema
disease|yes|fluid overload/heart failure
disease|yes|goiter
disease|yes|granulomatous disease
disease|yes|interstitial lung disease
disease|yes|lung cancer
disease|yes|pericardial effusion
disease|yes|pneumonia
nlp|yes|abnormal
nlp|yes|normal
technicalassessment|yes|artifact
technicalassessment|yes|breast/nipple shadows
technicalassessment|yes|low lung volumes
technicalassessment|yes|rotated
technicalassessment|yes|skin fold
tubesandlines|yes|chest port
tubesandlines|yes|chest tube
tubesandlines|yes|endotracheal tube
tubesandlines|yes|enteric tube
tubesandlines|yes|ij line
tubesandlines|yes|intra-aortic balloon pump
tubesandlines|yes|mediastinal drain
tubesandlines|yes|picc
tubesandlines|yes|pigtail cath

anatomicalfinding|yes|shoulder osteoarthritis
anatomicalfinding|yes|spinal degenerative changes
anatomicalfinding|yes|spinal fracture
anatomicalfinding|yes|sub-diaphragmatic air
anatomicalfinding|yes|subcutaneous air
anatomicalfinding|yes|superior mediastinal mass/enlargement
anatomicalfinding|yes|tortuous aorta
anatomicalfinding|yes|vascular calcification
anatomicalfinding|yes|vascular congestion
anatomicalfinding|yes|vascular redistribution
device|yes|aortic graft/repair
device|yes|cabg grafts
device|yes|cardiac pacer and wires
device|yes|prosthetic valve
disease|yes|alveolar hemorrhage
disease|yes|aspiration
disease|yes|copd/emphysema
disease|yes|fluid overload/heart failure
disease|yes|goiter
disease|yes|granulomatous disease
disease|yes|interstitial lung disease
disease|yes|lung cancer
disease|yes|pericardial effusion
disease|yes|pneumonia
nlp|yes|abnormal
nlp|yes|normal
technicalassessment|yes|artifact
technicalassessment|yes|breast/nipple shadows
technicalassessment|yes|low lung

...

technicalassessment|yes|low lung volumes
technicalassessment|yes|rotated
technicalassessment|yes|skin fold
tubesandlines|yes|chest port
tubesandlines|yes|chest tube
tubesandlines|yes|endotracheal tube
tubesandlines|yes|enteric tube
tubesandlines|yes|ij line
tubesandlines|yes|intra-aortic balloon pump
tubesandlines|yes|mediastinal drain
tubesandlines|yes|picc
tubesandlines|yes|pigtail catheter
tubesandlines|yes|subclavian line
tubesandlines|yes|swan-ganz catheter
tubesandlines|yes|tracheostomy tube
right costophrenic angle
anatomicalfinding|yes|airspace opacity
anatomicalfinding|yes|atelectasis
anatomicalfinding|yes|bone lesion
anatomicalfinding|yes|bronchiectasis
anatomicalfinding|yes|calcified nodule
anatomicalfinding|yes|clavicle fracture
anatomicalfinding|yes|consolidation
anatomicalfinding|yes|costophrenic angle blunting
anatomicalfinding|yes|cyst/bullae
anatomicalfinding|yes|diaphragmatic eventration (benign)
anatomicalfinding|yes|elevated hemidiaphragm
anatomicalfinding|yes|enlar

disease|yes|aspiration
disease|yes|copd/emphysema
disease|yes|fluid overload/heart failure
disease|yes|goiter
disease|yes|granulomatous disease
disease|yes|interstitial lung disease
disease|yes|lung cancer
disease|yes|pericardial effusion
disease|yes|pneumonia
nlp|yes|abnormal
nlp|yes|normal
technicalassessment|yes|artifact
technicalassessment|yes|breast/nipple shadows
technicalassessment|yes|low lung volumes
technicalassessment|yes|rotated
technicalassessment|yes|skin fold
tubesandlines|yes|chest port
tubesandlines|yes|chest tube
tubesandlines|yes|endotracheal tube
tubesandlines|yes|enteric tube
tubesandlines|yes|ij line
tubesandlines|yes|intra-aortic balloon pump
tubesandlines|yes|mediastinal drain
tubesandlines|yes|picc
tubesandlines|yes|pigtail catheter
tubesandlines|yes|subclavian line
tubesandlines|yes|swan-ganz catheter
tubesandlines|yes|tracheostomy tube
right lower lung zone
anatomicalfinding|yes|airspace opacity
anatomicalfinding|yes|atelectasis
anatomicalfinding|yes|bone les

anatomicalfinding|yes|tortuous aorta
anatomicalfinding|yes|vascular calcification
anatomicalfinding|yes|vascular congestion
anatomicalfinding|yes|vascular redistribution
device|yes|aortic graft/repair
device|yes|cabg grafts
device|yes|cardiac pacer and wires
device|yes|prosthetic valve
disease|yes|alveolar hemorrhage
disease|yes|aspiration
disease|yes|copd/emphysema
disease|yes|fluid overload/heart failure
disease|yes|goiter
disease|yes|granulomatous disease
disease|yes|interstitial lung disease
disease|yes|lung cancer
disease|yes|pericardial effusion
disease|yes|pneumonia
nlp|yes|abnormal
nlp|yes|normal
technicalassessment|yes|artifact
technicalassessment|yes|breast/nipple shadows
technicalassessment|yes|low lung volumes
technicalassessment|yes|rotated
technicalassessment|yes|skin fold
tubesandlines|yes|chest port
tubesandlines|yes|chest tube
tubesandlines|yes|endotracheal tube
tubesandlines|yes|enteric tube
tubesandlines|yes|ij line
tubesandlines|yes|intra-aortic balloon pump
tubesan

anatomicalfinding|yes|pulmonary edema/hazy opacity
anatomicalfinding|yes|rib fracture
anatomicalfinding|yes|scoliosis
anatomicalfinding|yes|shoulder osteoarthritis
anatomicalfinding|yes|spinal degenerative changes
anatomicalfinding|yes|spinal fracture
anatomicalfinding|yes|sub-diaphragmatic air
anatomicalfinding|yes|subcutaneous air
anatomicalfinding|yes|superior mediastinal mass/enlargement
anatomicalfinding|yes|tortuous aorta
anatomicalfinding|yes|vascular calcification
anatomicalfinding|yes|vascular congestion
anatomicalfinding|yes|vascular redistribution
device|yes|aortic graft/repair
device|yes|cabg grafts
device|yes|cardiac pacer and wires
device|yes|prosthetic valve
disease|yes|alveolar hemorrhage
disease|yes|aspiration
disease|yes|copd/emphysema
disease|yes|fluid overload/heart failure
disease|yes|goiter
disease|yes|granulomatous disease
disease|yes|interstitial lung disease
disease|yes|lung cancer
disease|yes|pericardial effusion
disease|yes|pneumonia
nlp|yes|abnormal
nlp|yes|

anatomicalfinding|yes|mass/nodule (not otherwise specified)
anatomicalfinding|yes|mediastinal displacement
anatomicalfinding|yes|mediastinal widening
anatomicalfinding|yes|multiple masses/nodules
anatomicalfinding|yes|pleural effusion
anatomicalfinding|yes|pleural/parenchymal scarring
anatomicalfinding|yes|pneumomediastinum
anatomicalfinding|yes|pneumothorax
anatomicalfinding|yes|pulmonary edema/hazy opacity
anatomicalfinding|yes|rib fracture
anatomicalfinding|yes|scoliosis
anatomicalfinding|yes|shoulder osteoarthritis
anatomicalfinding|yes|spinal degenerative changes
anatomicalfinding|yes|spinal fracture
anatomicalfinding|yes|sub-diaphragmatic air
anatomicalfinding|yes|subcutaneous air
anatomicalfinding|yes|superior mediastinal mass/enlargement
anatomicalfinding|yes|tortuous aorta
anatomicalfinding|yes|vascular calcification
anatomicalfinding|yes|vascular congestion
anatomicalfinding|yes|vascular redistribution
device|yes|aortic graft/repair
device|yes|cabg grafts
device|yes|cardiac p

In [18]:
dftally = pd.DataFrame(tally).copy()
dftally = dftally[dftally.freq_nlp!=0].reset_index(drop=True).copy()
print(dftally.shape)
dftally

(683, 4)


,bbox,attribute,freq_nlp,freq_has_bbox
0,abdomen,anatomicalfinding|yes|sub-diaphragmatic air,1247,1246
1,abdomen,nlp|yes|abnormal,5302,5296
2,abdomen,nlp|yes|normal,5088,5082
3,abdomen,technicalassessment|yes|artifact,21,21
4,abdomen,tubesandlines|yes|enteric tube,29138,29043
5,abdomen,tubesandlines|yes|intra-aortic balloon pump,20,20
6,abdomen,tubesandlines|yes|swan-ganz catheter,183,183
7,aortic arch,anatomicalfinding|yes|mediastinal widening,918,905
8,aortic arch,anatomicalfinding|yes|tortuous aorta,13892,13805
9,aortic arch,anatomicalfinding|yes|vascular calcification,9905,9853


In [19]:
# sanity check 
box = 'cardiac silhouette'
attr= 'anatomicalfinding|yes|lung opacity'
# this should be 0 
# in the chest imagenome dataset -- anatomical knowledge correctness is the rule
# even if the 'lung opacity' is described to be 'retrocardaic' (behind the heart), it is not assigned to be related to the heart anatomy
# 'lung opacity' is a finding of the 'lung' anatomy.
attributes[(attributes.bbox==box)&(attributes.attribute==attr)].shape

(0, 9)

In [21]:
print(dftally.shape)
dftally4paper = dftally.copy()
dftally4paper = dftally4paper[(dftally4paper.bbox.isin(objects))&(dftally4paper.attribute.isin(labels))].reset_index(drop=True).copy()
print(dftally4paper.shape)

(683, 4)
(611, 4)


In [22]:
dftally.to_csv('object_to_attribute_relation_stats.csv',index=False,encoding='utf-8')

In [24]:
# This format would be easier to use to generate labels for training for different tasks
# One can subset to a smaller set of objects and attributes (labels)

# drop bboxes with failed/no coordinates on images
print(attributes.shape)
attr4paper = attributes[(attributes.bbox.isin(objects))&(attributes.attribute.isin(labels))].reset_index(drop=True).copy()
print(attr4paper.shape)
attr4paper = attr4paper[attr4paper.bbox_coord_224 != "[0, 0, 0, 0]"].reset_index(drop=True).copy()
print(attr4paper.shape)
attr4paper = attr4paper[attr4paper.bbox_coord_224 != "[]"].reset_index(drop=True).copy()
print(attr4paper.shape)
attr4paper = attr4paper[~attr4paper.bbox_coord_224.str.contains("-")].reset_index(drop=True).copy()
print(attr4paper.shape)

(4529803, 9)
(4362649, 9)
(4326409, 9)
(4326409, 9)
(4326409, 9)


In [25]:
# for looking up bbox coordinates for the anatomical location from the scene graph json's objects field
attr4paper['object_id'] = ['_'.join([image_id,bbox]) for image_id,bbox in zip(attr4paper['image_id'],attr4paper['bbox'])]
attr4paper.head()

,study_id,image_id,bbox,attribute,bbox_coord_224,bbox_coord_original,relation,annot_id,label_name,object_id
0,55245226,000046e4-e4d7f796-72c3dba4-8b67a485-0eea211d,trachea,tubesandlines|yes|endotracheal tube,"[109, 75, 125, 96]","[1484, 762, 1702, 1048]",1,55245226|5|trachea|1|endotracheal tube,endotracheal tube,000046e4-e4d7f796-72c3dba4-8b67a485-0eea211d_t...
1,55245226,000046e4-e4d7f796-72c3dba4-8b67a485-0eea211d,left lower lung zone,nlp|yes|abnormal,"[132, 99, 210, 132]","[1797, 1089, 2859, 1538]",1,55245226|6|left lower lung zone|1|abnormal,abnormal,000046e4-e4d7f796-72c3dba4-8b67a485-0eea211d_l...
2,55245226,000046e4-e4d7f796-72c3dba4-8b67a485-0eea211d,left lower lung zone,anatomicalfinding|yes|atelectasis,"[132, 99, 210, 132]","[1797, 1089, 2859, 1538]",1,55245226|6|left lower lung zone|1|atelectasis,atelectasis,000046e4-e4d7f796-72c3dba4-8b67a485-0eea211d_l...
3,55245226,000046e4-e4d7f796-72c3dba4-8b67a485-0eea211d,left lower lung zone,anatomicalfinding|yes|lobar/segmental collapse,"[132, 99, 210, 132]","[1797, 1089, 2859, 1538]",1,55245226|6|left lower lung zone|1|lobar/segmen...,lobar/segmental collapse,000046e4-e4d7f796-72c3dba4-8b67a485-0eea211d_l...
4,55245226,000046e4-e4d7f796-72c3dba4-8b67a485-0eea211d,left lower lung zone,anatomicalfinding|yes|lung opacity,"[132, 99, 210, 132]","[1797, 1089, 2859, 1538]",1,55245226|6|left lower lung zone|1|lung opacity,lung opacity,000046e4-e4d7f796-72c3dba4-8b67a485-0eea211d_l...


In [14]:
# # Uncomment this cell to export

# # keep key columns needed for ML downstream tasks
# keep = ['image_id','bbox','attribute','bbox_coord_224','bbox_coord_original','object_id']
# attr4paper.to_csv(str(len(set(attr4paper.bbox)))+'objects_'+str(len(set(attr4paper.label_name)))+'attribute_relations_tabular_for_paper.txt',sep='\t',index=False,encoding='utf-8')


# Comparison relations statistics

In [16]:
# This table can also be generated from the scene_graph silver dataset using utils/postprocessing/scenegraph_postprocessing.py
# Make sure path to file is correct on your local machine

comp = pd.read_csv('../../../subset/scene_tabular/comparison_relations_tabular.txt',sep='\t')
print(comp.shape)
print(set(comp.comparison))
print(comp.loc[0,'relationship_id'])


(2090776, 11)
{'no change;;worsened', 'no change', 'improved;;worsened', 'improved;;no change;;worsened', 'improved', 'worsened', 'improved;;no change'}
50000052|5_50779042_C0507134;;C0018787_a09e5a6c_cf1ac468


In [17]:
for c in set(comp.comparison):
    print(c, len(comp[comp.comparison==c]))

# # These are usually hedging/uncertain statements from radiologists
# no change;;worsened 35348
# improved;;no change 49418
# # Have enough examples so probably should just exclude

# # These should probably be excluded as failures from the NLP pipeline (unable to assign the change relation correctly)
# improved;;no change;;worsened 3602
# improved;;worsened 36531

no change;;worsened 35325
no change 1069780
improved;;worsened 36531
improved;;no change;;worsened 3602
improved 340912
worsened 555212
improved;;no change 49414


In [18]:
neg = comp[comp.attribute.str.contains('|yes|',regex=False)].subject_id.tolist()
print(len(neg))
pos = comp[comp.attribute.str.contains('|no|',regex=False)].subject_id.tolist()
print(len(pos))
# could be for different attributes
print(len(set(neg).intersection(pos)))

1931964
158812
57318


In [19]:
# Similar rationale as that for picking the attribute annotations

# There may be multiple sents per bbox-comparison-attribute relation with different context for the attribute
comp['sent_loc'] = [float(x.split('_')[0].split('|')[-1]) for x in comp['relationship_id']]
comp['label_name'] = [x.split('|')[-1] for x in comp['attribute']]

# Sort by unique subject-object-label and sent_loc (order of sentences in a report)
comp = comp.sort_values(by=['subject_id','object_id','sent_loc','bbox','label_name']).reset_index(drop=True).copy()
print(comp.shape)

# keep last label context and comparison for subject-object-label combinations
comp = comp.drop_duplicates(subset=['subject_id','object_id','bbox','label_name'],keep='last').reset_index(drop=True).copy()
comp.drop(['sent_loc'],axis=1,inplace=True)
print(comp.shape)

(2090776, 13)
(1953277, 12)


In [20]:
neg = comp[comp.attribute.str.contains('|yes|',regex=False)].subject_id.tolist()
print(len(neg))
pos = comp[comp.attribute.str.contains('|no|',regex=False)].subject_id.tolist()
print(len(pos))

# could still be for different attributes
print(len(set(neg).intersection(pos)))

count = 0
for c in set(comp.comparison):
    ct = len(comp[comp.comparison==c])
    count = count + ct
    print(c, ct)
print('total', count)

1801198
152079
55104
no change;;worsened 30199
no change 1016431
improved;;worsened 30770
improved;;no change;;worsened 3112
improved 307751
worsened 524002
improved;;no change 41012
total 1953277


In [21]:
# # Check and See if there are still duplicated comparisons
# comp['comparison'] = comp.groupby(['subject_id','object_id','bbox','attribute'])['comparison'].transform(lambda x: ';;'.join(x))
# comp = comp.drop_duplicates().reset_index(drop=True).copy()

# print(comp.columns.tolist())
# print(comp.shape)
# print(set(comp.comparison))

# # Numbers are still the same as above so should be good now
# for c in set(comp.comparison):
#     print(c, len(comp[comp.comparison==c]))

In [22]:
# for grabbing the associated dicom or jpeg images from the MIMIC-CXR dataset
comp['current_image_id'] = [x.split('_')[0] for x in comp['subject_id']]
comp['previous_image_id'] = [x.split('_')[0] for x in comp['object_id']]

In [23]:
print(comp.loc[0,'relationship_id'])
comp.head()

59116935|6_54675277_C0225730_00005197_33e89953


,relationship_id,subject_id,object_id,bbox,comparison,attribute,sentence,bbox_coord_224_subject,bbox_coord_224_object,bbox_coord_original_subject,bbox_coord_original_object,label_name,current_image_id,previous_image_id
0,59116935|6_54675277_C0225730_00005197_33e89953,00005197-869d72f3-66210bf4-fa2c9d83-b613c4e7_l...,33e89953-a3344800-0b12cc28-ae13c39f-f350e654_l...,left lung,no change,nlp|yes|abnormal,remainder of the exam is unchanged noting bil...,"[118, 56, 191, 203]","[110, 55, 192, 200]","[1609, 504, 2605, 2510]","[1500, 491, 2619, 2469]",abnormal,00005197-869d72f3-66210bf4-fa2c9d83-b613c4e7,33e89953-a3344800-0b12cc28-ae13c39f-f350e654
1,59116935|6_54675277_C0225730_00005197_33e89953,00005197-869d72f3-66210bf4-fa2c9d83-b613c4e7_l...,33e89953-a3344800-0b12cc28-ae13c39f-f350e654_l...,left lung,no change,anatomicalfinding|yes|airspace opacity,remainder of the exam is unchanged noting bil...,"[118, 56, 191, 203]","[110, 55, 192, 200]","[1609, 504, 2605, 2510]","[1500, 491, 2619, 2469]",airspace opacity,00005197-869d72f3-66210bf4-fa2c9d83-b613c4e7,33e89953-a3344800-0b12cc28-ae13c39f-f350e654
2,59116935|6_54675277_C0225730_00005197_33e89953,00005197-869d72f3-66210bf4-fa2c9d83-b613c4e7_l...,33e89953-a3344800-0b12cc28-ae13c39f-f350e654_l...,left lung,no change,anatomicalfinding|yes|lung opacity,remainder of the exam is unchanged noting bil...,"[118, 56, 191, 203]","[110, 55, 192, 200]","[1609, 504, 2605, 2510]","[1500, 491, 2619, 2469]",lung opacity,00005197-869d72f3-66210bf4-fa2c9d83-b613c4e7,33e89953-a3344800-0b12cc28-ae13c39f-f350e654
3,59116935|6_54675277_C0225706_00005197_33e89953,00005197-869d72f3-66210bf4-fa2c9d83-b613c4e7_r...,33e89953-a3344800-0b12cc28-ae13c39f-f350e654_r...,right lung,no change,nlp|yes|abnormal,remainder of the exam is unchanged noting bil...,"[27, 55, 107, 194]","[20, 54, 110, 183]","[368, 491, 1459, 2387]","[272, 477, 1500, 2237]",abnormal,00005197-869d72f3-66210bf4-fa2c9d83-b613c4e7,33e89953-a3344800-0b12cc28-ae13c39f-f350e654
4,59116935|6_54675277_C0225706_00005197_33e89953,00005197-869d72f3-66210bf4-fa2c9d83-b613c4e7_r...,33e89953-a3344800-0b12cc28-ae13c39f-f350e654_r...,right lung,no change,anatomicalfinding|yes|airspace opacity,remainder of the exam is unchanged noting bil...,"[27, 55, 107, 194]","[20, 54, 110, 183]","[368, 491, 1459, 2387]","[272, 477, 1500, 2237]",airspace opacity,00005197-869d72f3-66210bf4-fa2c9d83-b613c4e7,33e89953-a3344800-0b12cc28-ae13c39f-f350e654


In [37]:
uniq_boxes = list(set(comp.bbox))
uniq_boxes.sort()
uniq_attrs = list(set(comp.attribute))
uniq_attrs.sort()
uniq_comps = list(set(comp.comparison))
uniq_comps.sort()
print(len(uniq_boxes))
print(len(uniq_attrs))
print(len(uniq_comps))
# checks for bbox coordinates - shouldn't have negatives
print(comp[comp.bbox_coord_224_subject.str.contains('-')].shape)
print(comp[comp.bbox_coord_224_object.str.contains('-')].shape)

38
112
7
(0, 14)
(0, 14)


In [38]:
# takes a while to run as there are many combinations
comptally = {'bbox':[],'comparison':[], 'attribute':[], 'freq_nlp':[], 'freq_have_bboxes':[]}
for box in uniq_boxes:
    print(box)
    for c in uniq_comps:
        print(c)
        for attr in uniq_attrs:
            print(attr)
            df = comp[(comp.bbox==box)&(comp.comparison==c)&(comp.attribute==attr)].reset_index(drop=True).copy()
            if len(df)>0:
                comptally['bbox'].append(box)
                comptally['comparison'].append(c)
                comptally['attribute'].append(attr)
                comptally['freq_nlp'].append(len(df))
                have_bboxes = df[(df.bbox_coord_224_subject != '[0, 0, 0, 0]')&(df.bbox_coord_224_object != '[0, 0, 0, 0]')]
                comptally['freq_have_bboxes'].append(len(have_bboxes))

abdomen
improved
anatomicalfinding|no|airspace opacity
anatomicalfinding|no|atelectasis
anatomicalfinding|no|bone lesion
anatomicalfinding|no|bronchiectasis
anatomicalfinding|no|calcified nodule
anatomicalfinding|no|consolidation
anatomicalfinding|no|cyst/bullae
anatomicalfinding|no|diaphragmatic eventration (benign)
anatomicalfinding|no|elevated hemidiaphragm
anatomicalfinding|no|enlarged cardiac silhouette
anatomicalfinding|no|enlarged hilum
anatomicalfinding|no|hernia
anatomicalfinding|no|hydropneumothorax
anatomicalfinding|no|hyperaeration
anatomicalfinding|no|increased reticular markings/ild pattern
anatomicalfinding|no|infiltration
anatomicalfinding|no|linear/patchy atelectasis
anatomicalfinding|no|lobar/segmental collapse
anatomicalfinding|no|lung lesion
anatomicalfinding|no|lung opacity
anatomicalfinding|no|mass/nodule (not otherwise specified)
anatomicalfinding|no|mediastinal displacement
anatomicalfinding|no|mediastinal widening
anatomicalfinding|no|multiple masses/nodules
an

disease|yes|granulomatous disease
disease|yes|interstitial lung disease
disease|yes|lung cancer
disease|yes|pericardial effusion
disease|yes|pneumonia
nlp|no|abnormal
nlp|no|normal
nlp|yes|abnormal
nlp|yes|normal
technicalassessment|yes|artifact
technicalassessment|yes|breast/nipple shadows
technicalassessment|yes|low lung volumes
technicalassessment|yes|rotated
technicalassessment|yes|skin fold
improved;;no change;;worsened
anatomicalfinding|no|airspace opacity
anatomicalfinding|no|atelectasis
anatomicalfinding|no|bone lesion
anatomicalfinding|no|bronchiectasis
anatomicalfinding|no|calcified nodule
anatomicalfinding|no|consolidation
anatomicalfinding|no|cyst/bullae
anatomicalfinding|no|diaphragmatic eventration (benign)
anatomicalfinding|no|elevated hemidiaphragm
anatomicalfinding|no|enlarged cardiac silhouette
anatomicalfinding|no|enlarged hilum
anatomicalfinding|no|hernia
anatomicalfinding|no|hydropneumothorax
anatomicalfinding|no|hyperaeration
anatomicalfinding|no|increased reticul

disease|no|aspiration
disease|no|copd/emphysema
disease|no|fluid overload/heart failure
disease|no|goiter
disease|no|granulomatous disease
disease|no|interstitial lung disease
disease|no|lung cancer
disease|no|pericardial effusion
disease|no|pneumonia
disease|yes|alveolar hemorrhage
disease|yes|aspiration
disease|yes|copd/emphysema
disease|yes|fluid overload/heart failure
disease|yes|goiter
disease|yes|granulomatous disease
disease|yes|interstitial lung disease
disease|yes|lung cancer
disease|yes|pericardial effusion
disease|yes|pneumonia
nlp|no|abnormal
nlp|no|normal
nlp|yes|abnormal
nlp|yes|normal
technicalassessment|yes|artifact
technicalassessment|yes|breast/nipple shadows
technicalassessment|yes|low lung volumes
technicalassessment|yes|rotated
technicalassessment|yes|skin fold
no change
anatomicalfinding|no|airspace opacity
anatomicalfinding|no|atelectasis
anatomicalfinding|no|bone lesion
anatomicalfinding|no|bronchiectasis
anatomicalfinding|no|calcified nodule
anatomicalfinding|n

anatomicalfinding|yes|spinal degenerative changes
anatomicalfinding|yes|spinal fracture
anatomicalfinding|yes|sub-diaphragmatic air
anatomicalfinding|yes|subcutaneous air
anatomicalfinding|yes|superior mediastinal mass/enlargement
anatomicalfinding|yes|tortuous aorta
anatomicalfinding|yes|vascular calcification
anatomicalfinding|yes|vascular congestion
anatomicalfinding|yes|vascular redistribution
disease|no|alveolar hemorrhage
disease|no|aspiration
disease|no|copd/emphysema
disease|no|fluid overload/heart failure
disease|no|goiter
disease|no|granulomatous disease
disease|no|interstitial lung disease
disease|no|lung cancer
disease|no|pericardial effusion
disease|no|pneumonia
disease|yes|alveolar hemorrhage
disease|yes|aspiration
disease|yes|copd/emphysema
disease|yes|fluid overload/heart failure
disease|yes|goiter
disease|yes|granulomatous disease
disease|yes|interstitial lung disease
disease|yes|lung cancer
disease|yes|pericardial effusion
disease|yes|pneumonia
nlp|no|abnormal
nlp|no|

anatomicalfinding|yes|mediastinal widening
anatomicalfinding|yes|multiple masses/nodules
anatomicalfinding|yes|pleural effusion
anatomicalfinding|yes|pleural/parenchymal scarring
anatomicalfinding|yes|pneumomediastinum
anatomicalfinding|yes|pneumothorax
anatomicalfinding|yes|pulmonary edema/hazy opacity
anatomicalfinding|yes|rib fracture
anatomicalfinding|yes|scoliosis
anatomicalfinding|yes|shoulder osteoarthritis
anatomicalfinding|yes|spinal degenerative changes
anatomicalfinding|yes|spinal fracture
anatomicalfinding|yes|sub-diaphragmatic air
anatomicalfinding|yes|subcutaneous air
anatomicalfinding|yes|superior mediastinal mass/enlargement
anatomicalfinding|yes|tortuous aorta
anatomicalfinding|yes|vascular calcification
anatomicalfinding|yes|vascular congestion
anatomicalfinding|yes|vascular redistribution
disease|no|alveolar hemorrhage
disease|no|aspiration
disease|no|copd/emphysema
disease|no|fluid overload/heart failure
disease|no|goiter
disease|no|granulomatous disease
disease|no|

...

anatomicalfinding|yes|diaphragmatic eventration (benign)
anatomicalfinding|yes|elevated hemidiaphragm
anatomicalfinding|yes|enlarged cardiac silhouette
anatomicalfinding|yes|enlarged hilum
anatomicalfinding|yes|hernia
anatomicalfinding|yes|hydropneumothorax
anatomicalfinding|yes|hyperaeration
anatomicalfinding|yes|increased reticular markings/ild pattern
anatomicalfinding|yes|infiltration
anatomicalfinding|yes|linear/patchy atelectasis
anatomicalfinding|yes|lobar/segmental collapse
anatomicalfinding|yes|lung lesion
anatomicalfinding|yes|lung opacity
anatomicalfinding|yes|mass/nodule (not otherwise specified)
anatomicalfinding|yes|mediastinal displacement
anatomicalfinding|yes|mediastinal widening
anatomicalfinding|yes|multiple masses/nodules
anatomicalfinding|yes|pleural effusion
anatomicalfinding|yes|pleural/parenchymal scarring
anatomicalfinding|yes|pneumomediastinum
anatomicalfinding|yes|pneumothorax
anatomicalfinding|yes|pulmonary edema/hazy opacity
anatomicalfinding|yes|rib fractu

anatomicalfinding|no|vascular redistribution
anatomicalfinding|yes|airspace opacity
anatomicalfinding|yes|atelectasis
anatomicalfinding|yes|bone lesion
anatomicalfinding|yes|bronchiectasis
anatomicalfinding|yes|calcified nodule
anatomicalfinding|yes|clavicle fracture
anatomicalfinding|yes|consolidation
anatomicalfinding|yes|costophrenic angle blunting
anatomicalfinding|yes|cyst/bullae
anatomicalfinding|yes|diaphragmatic eventration (benign)
anatomicalfinding|yes|elevated hemidiaphragm
anatomicalfinding|yes|enlarged cardiac silhouette
anatomicalfinding|yes|enlarged hilum
anatomicalfinding|yes|hernia
anatomicalfinding|yes|hydropneumothorax
anatomicalfinding|yes|hyperaeration
anatomicalfinding|yes|increased reticular markings/ild pattern
anatomicalfinding|yes|infiltration
anatomicalfinding|yes|linear/patchy atelectasis
anatomicalfinding|yes|lobar/segmental collapse
anatomicalfinding|yes|lung lesion
anatomicalfinding|yes|lung opacity
anatomicalfinding|yes|mass/nodule (not otherwise specifi

anatomicalfinding|no|rib fracture
anatomicalfinding|no|scoliosis
anatomicalfinding|no|spinal degenerative changes
anatomicalfinding|no|spinal fracture
anatomicalfinding|no|sub-diaphragmatic air
anatomicalfinding|no|subcutaneous air
anatomicalfinding|no|superior mediastinal mass/enlargement
anatomicalfinding|no|tortuous aorta
anatomicalfinding|no|vascular calcification
anatomicalfinding|no|vascular congestion
anatomicalfinding|no|vascular redistribution
anatomicalfinding|yes|airspace opacity
anatomicalfinding|yes|atelectasis
anatomicalfinding|yes|bone lesion
anatomicalfinding|yes|bronchiectasis
anatomicalfinding|yes|calcified nodule
anatomicalfinding|yes|clavicle fracture
anatomicalfinding|yes|consolidation
anatomicalfinding|yes|costophrenic angle blunting
anatomicalfinding|yes|cyst/bullae
anatomicalfinding|yes|diaphragmatic eventration (benign)
anatomicalfinding|yes|elevated hemidiaphragm
anatomicalfinding|yes|enlarged cardiac silhouette
anatomicalfinding|yes|enlarged hilum
anatomicalf

anatomicalfinding|no|mass/nodule (not otherwise specified)
anatomicalfinding|no|mediastinal displacement
anatomicalfinding|no|mediastinal widening
anatomicalfinding|no|multiple masses/nodules
anatomicalfinding|no|pleural effusion
anatomicalfinding|no|pleural/parenchymal scarring
anatomicalfinding|no|pneumomediastinum
anatomicalfinding|no|pneumothorax
anatomicalfinding|no|pulmonary edema/hazy opacity
anatomicalfinding|no|rib fracture
anatomicalfinding|no|scoliosis
anatomicalfinding|no|spinal degenerative changes
anatomicalfinding|no|spinal fracture
anatomicalfinding|no|sub-diaphragmatic air
anatomicalfinding|no|subcutaneous air
anatomicalfinding|no|superior mediastinal mass/enlargement
anatomicalfinding|no|tortuous aorta
anatomicalfinding|no|vascular calcification
anatomicalfinding|no|vascular congestion
anatomicalfinding|no|vascular redistribution
anatomicalfinding|yes|airspace opacity
anatomicalfinding|yes|atelectasis
anatomicalfinding|yes|bone lesion
anatomicalfinding|yes|bronchiecta

anatomicalfinding|no|enlarged hilum
anatomicalfinding|no|hernia
anatomicalfinding|no|hydropneumothorax
anatomicalfinding|no|hyperaeration
anatomicalfinding|no|increased reticular markings/ild pattern
anatomicalfinding|no|infiltration
anatomicalfinding|no|linear/patchy atelectasis
anatomicalfinding|no|lobar/segmental collapse
anatomicalfinding|no|lung lesion
anatomicalfinding|no|lung opacity
anatomicalfinding|no|mass/nodule (not otherwise specified)
anatomicalfinding|no|mediastinal displacement
anatomicalfinding|no|mediastinal widening
anatomicalfinding|no|multiple masses/nodules
anatomicalfinding|no|pleural effusion
anatomicalfinding|no|pleural/parenchymal scarring
anatomicalfinding|no|pneumomediastinum
anatomicalfinding|no|pneumothorax
anatomicalfinding|no|pulmonary edema/hazy opacity
anatomicalfinding|no|rib fracture
anatomicalfinding|no|scoliosis
anatomicalfinding|no|spinal degenerative changes
anatomicalfinding|no|spinal fracture
anatomicalfinding|no|sub-diaphragmatic air
anatomica

In [39]:
dfcomptally = pd.DataFrame(comptally).copy()
dfcomptally = dfcomptally[dfcomptally.freq_nlp!=0].reset_index(drop=True).copy()
print(dfcomptally.shape)
dfcomptally

(4273, 5)


,bbox,comparison,attribute,freq_nlp,freq_have_bboxes
0,abdomen,improved,anatomicalfinding|no|sub-diaphragmatic air,50,50
1,abdomen,improved,anatomicalfinding|yes|sub-diaphragmatic air,92,92
2,abdomen,improved,nlp|no|abnormal,50,50
3,abdomen,improved,nlp|yes|abnormal,215,215
4,abdomen,improved,nlp|yes|normal,12,12
5,abdomen,improved;;no change,anatomicalfinding|no|sub-diaphragmatic air,1,1
6,abdomen,improved;;no change,anatomicalfinding|yes|sub-diaphragmatic air,4,4
7,abdomen,improved;;no change,nlp|no|abnormal,2,2
8,abdomen,improved;;no change,nlp|yes|abnormal,8,8
9,abdomen,improved;;worsened,nlp|yes|abnormal,7,7


In [40]:
print(dfcomptally.shape)
# For paper -- keep to just objects that could have bbox coordinates
# again, could subset the objects and or attributes further if want to simply the problem.
dfcomptally4paper = dfcomptally.copy()
dfcomptally4paper = dfcomptally4paper[(dfcomptally4paper.bbox.isin(objects))&(dfcomptally4paper.attribute.isin(labels))].reset_index(drop=True).copy()
print(dfcomptally4paper.shape)

(4273, 5)
(2681, 5)


In [41]:
dfcomptally4paper.to_csv('object_to_object_comparison_relation_stats.csv',index=False,encoding='utf-8')

# Comparison bbox statistics irrespective of attribute

In [24]:
print(comp.shape)
# irrespective of attribute
comp4baseline = comp.drop(['attribute','label_name','sentence'],axis=1).drop_duplicates().reset_index(drop=True).copy()
print(comp4baseline.shape)
# drop failed bboxes so keep only objects with bbox coordinates on associated images
comp4baseline = comp4baseline[comp4baseline.bbox_coord_224_subject!='[0, 0, 0, 0]'].copy()
comp4baseline = comp4baseline[comp4baseline.bbox_coord_224_object!='[0, 0, 0, 0]'].reset_index(drop=True).copy()
print(comp4baseline.shape)
comp4baseline = comp4baseline[comp4baseline.bbox.isin(objects)].copy()
print('all 29 objects', comp4baseline.shape) # number to report in paper abstract
comp4baseline['comparison'] = comp4baseline.groupby(['subject_id','object_id','bbox'])['comparison'].transform(lambda x: ';;'.join(x))
comp4baseline['comparison'] = [';;'.join(sorted(list(set(x.split(';;'))))) for x in comp4baseline['comparison']]
comp4baseline = comp4baseline.drop_duplicates().reset_index(drop=True).copy()


# Get number of unique pairs of subject-object comparison relations (image-level) irrespective of attribute
# i.e., just asking whether an anatomy has improved, worsened or had no change
print(comp4baseline.shape)
print(set(comp4baseline.comparison))
comp4baseline.head()

(1953277, 14)
(702264, 11)
(678543, 11)
all 29 objects (678543, 11)
(678543, 11)
{'no change;;worsened', 'no change', 'improved;;worsened', 'improved;;no change;;worsened', 'improved', 'worsened', 'improved;;no change'}


,relationship_id,subject_id,object_id,bbox,comparison,bbox_coord_224_subject,bbox_coord_224_object,bbox_coord_original_subject,bbox_coord_original_object,current_image_id,previous_image_id
0,59116935|6_54675277_C0225730_00005197_33e89953,00005197-869d72f3-66210bf4-fa2c9d83-b613c4e7_l...,33e89953-a3344800-0b12cc28-ae13c39f-f350e654_l...,left lung,no change,"[118, 56, 191, 203]","[110, 55, 192, 200]","[1609, 504, 2605, 2510]","[1500, 491, 2619, 2469]",00005197-869d72f3-66210bf4-fa2c9d83-b613c4e7,33e89953-a3344800-0b12cc28-ae13c39f-f350e654
1,59116935|6_54675277_C0225706_00005197_33e89953,00005197-869d72f3-66210bf4-fa2c9d83-b613c4e7_r...,33e89953-a3344800-0b12cc28-ae13c39f-f350e654_r...,right lung,no change,"[27, 55, 107, 194]","[20, 54, 110, 183]","[368, 491, 1459, 2387]","[272, 477, 1500, 2237]",00005197-869d72f3-66210bf4-fa2c9d83-b613c4e7,33e89953-a3344800-0b12cc28-ae13c39f-f350e654
2,50189753|6_58204679_C0507134;;C0018787_0000d3b...,0000d3be-591ae3b7-b03a7497-8319c02b-650bb4ab_c...,a496ead6-651d5823-4c81964a-55e28b32-41ca38d6_c...,cardiac silhouette,no change,"[84, 110, 171, 173]","[79, 100, 167, 165]","[886, 1500, 2073, 2360]","[818, 1364, 2019, 2251]",0000d3be-591ae3b7-b03a7497-8319c02b-650bb4ab,a496ead6-651d5823-4c81964a-55e28b32-41ca38d6
3,56025085|9_51318109_C0934571_00010923_9763742c,00010923-3e9327ed-01db276b-5872bf82-5c844e7f_l...,9763742c-618dfa0e-4c404afe-7bf3ce1a-027c0e53_l...,left lower lung zone,improved,"[112, 123, 199, 173]","[123, 130, 207, 182]","[1467, 1415, 2607, 2070]","[1678, 1514, 2824, 2223]",00010923-3e9327ed-01db276b-5872bf82-5c844e7f,9763742c-618dfa0e-4c404afe-7bf3ce1a-027c0e53
4,56025085|7_51318109_C0225730_00010923_9763742c,00010923-3e9327ed-01db276b-5872bf82-5c844e7f_l...,9763742c-618dfa0e-4c404afe-7bf3ce1a-027c0e53_l...,left lung,improved;;no change,"[107, 25, 199, 173]","[123, 37, 207, 182]","[1401, 131, 2607, 2070]","[1678, 245, 2824, 2223]",00010923-3e9327ed-01db276b-5872bf82-5c844e7f,9763742c-618dfa0e-4c404afe-7bf3ce1a-027c0e53


In [25]:
pairs = [x.split('_')[0]+y.split('_')[0] for x,y in zip(comp4baseline['subject_id'],comp4baseline['object_id'])]
print('Pairs of comparison images', len(set(pairs)))

Pairs of comparison images 128468


## Example 1 - Preparing labels for the edema and heart failure comparison problem

In [26]:
# For the dataset paper, let's just focus on labels related to pulmonary edema and heart failure
task = ['pulmonary edema/hazy opacity','fluid overload/heart failure']

comp4paper = comp[comp.label_name.isin(task)].reset_index(drop=True).copy()
print(comp4paper.shape)

print(len(set(comp4paper.relationship_id)))
print('attribute', set(comp4paper.attribute))
print('bbox', set(comp4paper.bbox))

(141182, 14)
139158
attribute {'anatomicalfinding|no|pulmonary edema/hazy opacity', 'disease|no|fluid overload/heart failure', 'disease|yes|fluid overload/heart failure', 'anatomicalfinding|yes|pulmonary edema/hazy opacity'}
bbox {'left mid lung zone', 'right mid lung zone', 'left costophrenic angle', 'right upper lung zone', 'left lung', 'left hilar structures', 'cardiac silhouette', 'right costophrenic angle', 'right lower lung zone', 'left lower lung zone', 'left upper lung zone', 'right lung', 'right hilar structures'}


In [27]:
comp4paper.head()

,relationship_id,subject_id,object_id,bbox,comparison,attribute,sentence,bbox_coord_224_subject,bbox_coord_224_object,bbox_coord_original_subject,bbox_coord_original_object,label_name,current_image_id,previous_image_id
0,55547878|9_54997993_C1708367_00054ff1_0b301cfb,00054ff1-47f5c5c2-e8dc7c52-1deafa05-e1174cd0_l...,0b301cfb-27ddc5f3-5f81f78c-badcee86-e1048916_l...,left hilar structures,no change,anatomicalfinding|yes|pulmonary edema/hazy opa...,layering right pleural effusion and associate...,"[116, 83, 153, 137]","[124, 105, 164, 140]","[1538, 888, 2029, 1604]","[1688, 1170, 2233, 1647]",pulmonary edema/hazy opacity,00054ff1-47f5c5c2-e8dc7c52-1deafa05-e1174cd0,0b301cfb-27ddc5f3-5f81f78c-badcee86-e1048916
1,55547878|9_54997993_C0225730_00054ff1_0b301cfb,00054ff1-47f5c5c2-e8dc7c52-1deafa05-e1174cd0_l...,0b301cfb-27ddc5f3-5f81f78c-badcee86-e1048916_l...,left lung,no change,anatomicalfinding|yes|pulmonary edema/hazy opa...,layering right pleural effusion and associate...,"[115, 37, 196, 198]","[122, 56, 221, 180]","[1525, 278, 2599, 2413]","[1661, 503, 3009, 2192]",pulmonary edema/hazy opacity,00054ff1-47f5c5c2-e8dc7c52-1deafa05-e1174cd0,0b301cfb-27ddc5f3-5f81f78c-badcee86-e1048916
2,55547878|9_54997993_C1708369_00054ff1_0b301cfb,00054ff1-47f5c5c2-e8dc7c52-1deafa05-e1174cd0_r...,0b301cfb-27ddc5f3-5f81f78c-badcee86-e1048916_r...,right hilar structures,no change,anatomicalfinding|yes|pulmonary edema/hazy opa...,layering right pleural effusion and associate...,"[67, 85, 106, 122]","[80, 105, 120, 140]","[888, 915, 1405, 1405]","[1089, 1170, 1633, 1647]",pulmonary edema/hazy opacity,00054ff1-47f5c5c2-e8dc7c52-1deafa05-e1174cd0,0b301cfb-27ddc5f3-5f81f78c-badcee86-e1048916
3,55547878|9_54997993_C0225706_00054ff1_0b301cfb,00054ff1-47f5c5c2-e8dc7c52-1deafa05-e1174cd0_r...,0b301cfb-27ddc5f3-5f81f78c-badcee86-e1048916_r...,right lung,no change,anatomicalfinding|yes|pulmonary edema/hazy opa...,layering right pleural effusion and associate...,"[23, 32, 109, 163]","[23, 56, 122, 180]","[305, 212, 1445, 1949]","[313, 503, 1661, 2192]",pulmonary edema/hazy opacity,00054ff1-47f5c5c2-e8dc7c52-1deafa05-e1174cd0,0b301cfb-27ddc5f3-5f81f78c-badcee86-e1048916
4,54084691|12_57692900_C0507134;;C0018787_000c51...,000c51b9-e249dd2e-33b7c5c3-b85b2abb-c66a30b8_c...,2d0b85d8-705efde9-61099cd4-0c01ed46-e012a16e_c...,cardiac silhouette,worsened,disease|yes|fluid overload/heart failure,"impression: 1. progression of chf findings, n...","[76, 98, 159, 168]","[74, 80, 152, 146]","[820, 1218, 1851, 2088]","[748, 1089, 1810, 1987]",fluid overload/heart failure,000c51b9-e249dd2e-33b7c5c3-b85b2abb-c66a30b8,2d0b85d8-705efde9-61099cd4-0c01ed46-e012a16e


In [28]:
print(set(comp4paper.comparison))
print()
for c in set(comp4paper.comparison):
    print(c, len(comp4paper[comp4paper.comparison==c]))

{'no change;;worsened', 'no change', 'improved;;worsened', 'improved;;no change;;worsened', 'improved', 'worsened', 'improved;;no change'}

no change;;worsened 2832
no change 48581
improved;;worsened 2804
improved;;no change;;worsened 388
improved 35420
worsened 46808
improved;;no change 4349


In [32]:
# Check any contradictions?
lab = task[1]
neg = comp4paper[(comp4paper.label_name==lab)&(comp4paper.attribute.str.contains('|no|',regex=False))].subject_id.tolist()
print(len(neg))
pos = comp4paper[(comp4paper.label_name==lab)&(comp4paper.attribute.str.contains('|yes|',regex=False))].subject_id.tolist()
print(len(pos))
# Should be 0
print(len(set(neg).intersection(pos)))

1104
9868
0


In [33]:
# if matches == good
print(comp4paper[comp4paper.attribute.str.contains(lab,regex=False)].shape)
print(len(set(comp4paper[comp4paper.attribute.str.contains(lab,regex=False)].relationship_id)))

(10972, 14)
10972


In [34]:
# This id is only unique to each subject-object-comparison relation combination 
# -- and not unique if ones adds the different possible attributes the comparison can be referring to
print(comp4paper.loc[0,'relationship_id'])

55547878|9_54997993_C1708367_00054ff1_0b301cfb


In [35]:
print(comp4paper.drop_duplicates(subset=['relationship_id','subject_id','object_id','bbox','comparison','attribute']).shape)
# relationshp id is not unique to attribute level but is unique to other parameters
print(comp4paper.drop_duplicates(subset=['relationship_id','subject_id','object_id','bbox','comparison']).shape)
print(comp4paper.drop_duplicates(subset=['relationship_id','subject_id','object_id','bbox','attribute']).shape)
print(comp4paper.drop_duplicates(subset=['relationship_id','subject_id','object_id','comparison','attribute']).shape)
print(comp4paper.drop_duplicates(subset=['relationship_id','subject_id','bbox','comparison','attribute']).shape)
print(comp4paper.drop_duplicates(subset=['relationship_id','object_id','bbox','comparison','attribute']).shape)
print(comp4paper.drop_duplicates(subset=['subject_id','object_id','bbox','comparison','attribute']).shape)

(141182, 14)
(139158, 14)
(141182, 14)
(141182, 14)
(141182, 14)
(141182, 14)
(141182, 14)


In [46]:
print(set(comp4paper.comparison))
print()
# for c in set(comp4paper.comparison):
for c in ['improved','worsened','no change']:
    for b in ['right lung','left lung','cardiac silhouette']:
        print(b,'-', c,': ', len(comp4paper[(comp4paper.comparison==c)&(comp4paper.bbox==b)]))

{'no change;;worsened', 'no change', 'improved;;worsened', 'improved;;no change;;worsened', 'improved', 'worsened', 'improved;;no change'}

right lung - improved :  7984
left lung - improved :  7986
cardiac silhouette - improved :  427
right lung - worsened :  10037
left lung - worsened :  10036
cardiac silhouette - worsened :  1121
right lung - no change :  10667
left lung - no change :  10668
cardiac silhouette - no change :  1167


In [47]:
# For baseline, we will just run experiment for the key anatomies for pulmonary edema and heart failure
# for a more balanced dataset, we will keep to left lung and right lung
# the 'no change' relation is not always described in reports and technically identifying no change is hard for medical images
# 'no change' of anatomy is obvious to radiology experts but they tell this by considering many other imaging, positional and clinical factors
# We tried experiments with 'no change' as a target but it is very hard to get right
# So for baseline example, we simplified the comparison problem to just 'improved' and 'worsened' classification for
# the 'left lung' and 'right lung' anatomies
# future work is being done to improve on architectures that can handle change relations across images better

comp4paper = comp4paper[(comp4paper.comparison.isin(['improved','worsened']))
                        &(comp4paper.bbox.isin(['right lung','left lung']))].reset_index(drop=True).copy()
print(comp4paper.shape)

(36043, 14)


In [28]:
# # uncomment to export
# comp4paper.to_csv('edema_chf_comparison_relations_tabular_for_paper.txt',sep='\t',index=False,encoding='utf-8')

# Plots & exploratory stats

## For object-to-attribute relations

In [36]:
# Read in previous exported file
attr_tally = pd.read_csv('object_to_attribute_relation_stats.csv')
print(attr_tally.shape)
attr_tally.head()

(683, 4)


,bbox,attribute,freq_nlp,freq_has_bbox
0,abdomen,anatomicalfinding|yes|sub-diaphragmatic air,1247,1246
1,abdomen,nlp|yes|abnormal,5302,5296
2,abdomen,nlp|yes|normal,5088,5082
3,abdomen,technicalassessment|yes|artifact,21,21
4,abdomen,tubesandlines|yes|enteric tube,29138,29043


In [37]:
# Group to bbox object level
attr_tally2 = attr_tally.copy()
attr_tally2['freq_nlp'] = attr_tally2.groupby(['bbox'])['freq_nlp'].transform(lambda x: sum(x))
attr_tally2['freq_has_bbox'] = attr_tally2.groupby(['bbox'])['freq_has_bbox'].transform(lambda x: sum(x))
print(attr_tally2.shape)
attr_tally2 = attr_tally2.drop(['attribute'],axis=1).drop_duplicates().reset_index(drop=True).copy()
print(attr_tally2.shape)

# For reporting in paper
attr_tally2['% missing'] = [round(1-x/y,4) for x,y in zip(attr_tally2['freq_has_bbox'],attr_tally2['freq_nlp'])]

(683, 4)
(38, 3)


In [38]:
attr_tally2[attr_tally2.bbox.isin(objects)].sort_values(by=['freq_nlp'],ascending=False)

,bbox,freq_nlp,freq_has_bbox,% missing
14,left lung,753338,753125,0.0003
30,right lung,737810,737513,0.0004
13,left lower lung zone,332072,324245,0.0236
29,right lower lung zone,291022,284421,0.0227
2,cardiac silhouette,285252,285227,0.0001
28,right hilar structures,269889,264721,0.0191
12,left hilar structures,262391,256398,0.0228
18,mediastinum,257122,257069,0.0002
10,left costophrenic angle,205128,203828,0.0063
26,right costophrenic angle,197895,197125,0.0039


## For object-to-object comparison relations

In [39]:
comp_tally = pd.read_csv('object_to_object_comparison_relation_stats.csv')
print(comp_tally.shape)
comp_tally.head()

(2681, 5)


,bbox,comparison,attribute,freq_nlp,freq_have_bboxes
0,abdomen,improved,anatomicalfinding|yes|sub-diaphragmatic air,92,92
1,abdomen,improved,nlp|yes|abnormal,215,215
2,abdomen,improved,nlp|yes|normal,12,12
3,abdomen,improved;;no change,anatomicalfinding|yes|sub-diaphragmatic air,4,4
4,abdomen,improved;;no change,nlp|yes|abnormal,8,8


In [40]:
print('Total number of relations from report:', sum(comp_tally.freq_nlp.tolist()))
print('Total number of relations have bboxes:', sum(comp_tally.freq_have_bboxes.tolist()))

Total number of relations from report: 1784077
Total number of relations have bboxes: 1742881


In [41]:
# ignoring type of object-attribute relation (affirmed vs negated) for the comparison relation
comp_tally2 = comp_tally.copy()
comp_tally2['label_name'] = [x.split('|')[-1] for x in comp_tally2['attribute']]
comp_tally2['freq_nlp'] = comp_tally2.groupby(['bbox','comparison','label_name'])['freq_nlp'].transform(lambda x: sum(x))
comp_tally2['freq_have_bboxes'] = comp_tally2.groupby(['bbox','comparison','label_name'])['freq_have_bboxes'].transform(lambda x: sum(x))
keep = ['bbox','comparison','label_name','freq_nlp','freq_have_bboxes']
comp_tally2 = comp_tally2[keep].drop_duplicates().reset_index(drop=True).copy()
print(comp_tally2.shape)
comp_tally2.head()

(2681, 5)


,bbox,comparison,label_name,freq_nlp,freq_have_bboxes
0,abdomen,improved,sub-diaphragmatic air,92,92
1,abdomen,improved,abnormal,215,215
2,abdomen,improved,normal,12,12
3,abdomen,improved;;no change,sub-diaphragmatic air,4,4
4,abdomen,improved;;no change,abnormal,8,8


In [42]:
print(set(comp_tally2.comparison))
targets = ['improved'
           , 'worsened'
           , 'no change']

{'no change;;worsened', 'improved;;worsened', 'no change', 'improved;;no change;;worsened', 'improved', 'worsened', 'improved;;no change'}


In [43]:
task = ['lung opacity']

In [44]:
comp_exp = comp_tally2[(comp_tally2.label_name.isin(task))&(comp_tally2.comparison.isin(targets))].reset_index(drop=True).copy()
comp_exp.sort_values(by='freq_have_bboxes',ascending=False)

,bbox,comparison,label_name,freq_nlp,freq_have_bboxes
13,left lung,no change,lung opacity,39371,39336
34,right lung,no change,lung opacity,38046,38014
35,right lung,worsened,lung opacity,24748,24728
14,left lung,worsened,lung opacity,24491,24472
10,left lower lung zone,no change,lung opacity,19808,18648
4,left costophrenic angle,no change,lung opacity,18210,17989
25,right costophrenic angle,no change,lung opacity,17370,17185
31,right lower lung zone,no change,lung opacity,16437,15508
33,right lung,improved,lung opacity,15161,15152
12,left lung,improved,lung opacity,14346,14339


In [45]:
comp_exp

,bbox,comparison,label_name,freq_nlp,freq_have_bboxes
0,left apical zone,improved,lung opacity,173,165
1,left apical zone,no change,lung opacity,1711,1618
2,left apical zone,worsened,lung opacity,299,277
3,left costophrenic angle,improved,lung opacity,4780,4699
4,left costophrenic angle,no change,lung opacity,18210,17989
5,left costophrenic angle,worsened,lung opacity,9654,9524
6,left hilar structures,improved,lung opacity,8214,7683
7,left hilar structures,no change,lung opacity,12471,11788
8,left hilar structures,worsened,lung opacity,11331,10727
9,left lower lung zone,improved,lung opacity,6615,6235
